# Hello There!

## Welcome to your Jupyter Notebook kickstarter!

---

In [1]:
# Hi there! I'm a code cell and what you're reading now is just an ordinary python comment
# here you can write all the code you want and just execute it by pressing Shift+Enter
# Let's figure out what the time and date are

import time

time_now = time.ctime()

print("The date and time now are {}".format(time_now))

# you'll see the content of stdout under me after you execute me

The date and time now are Sun Dec  7 15:08:25 2025


---